In [15]:
PROJECT= ! (gcloud config get-value project) 
BUCKET_NAME = PROJECT[0] + "-bucket"
INPUT_NAME="data.jsonl"
OUTPUT_DIR="results"
OUTPUT_FILE="results.jsonl"

In [ ]:
prompt="hello"

In [ ]:
import json
import os
from google.cloud import storage

test = [{"prompt":"hello"}]

with open(INPUT_NAME, 'w') as f:
    for item in test:
        f.write(json.dumps(item) + "\n")

with open(INPUT_NAME) as f:
    for line in f:
        print(line)
        
# Get the bucket name
bucket_name = BUCKET_NAME

# Get the file path
file_path = INPUT_NAME

# Create a storage client
client = storage.Client()
bucket = client.bucket(bucket_name)

# Upload the file
bucket.blob(file_path).upload_from_filename(file_path)

In [ ]:
from vertexai.preview.language_models import TextGenerationModel
text_model = TextGenerationModel.from_pretrained("text-bison")
batch_prediction_job = text_model.batch_predict(
  dataset=["gs://"+BUCKET_NAME+"/"+INPUT_NAME],
  destination_uri_prefix="gs://"+BUCKET_NAME+"/"+OUTPUT_DIR,
  # Optional:
  model_parameters={
      "maxOutputTokens": "200",
      "temperature": "0.2",
      "topP": "0.95",
      "topK": "40",
  },
)
print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)

In [ ]:
batch_prediction_job.output_info.gcs_output_directory.replace("gs://"+BUCKET_NAME+"/","")

In [ ]:
blobs = bucket.list_blobs(prefix=batch_prediction_job.output_info.gcs_output_directory.replace("gs://"+BUCKET_NAME+"/",""))

for blob in blobs:
    print(blob.name)
    blob.download_to_filename(OUTPUT_FILE)

In [ ]:
import json

with open(OUTPUT_FILE, 'r') as f:
  response = json.load(f)

response["predictions"][0]["content"]